In [111]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

np.set_printoptions(precision=3, suppress=True)

In [112]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.9.1


In [113]:
nickel_data = pd.read_csv("dataWithHeat.csv", index_col=0)

# For now we must get rid of the data without heat treatments.
# Luckily, this is shown in values with zero heat treatment 1 temp
nickel_data = nickel_data.drop(
    nickel_data[nickel_data['Heat treatment 1 Temperature'] < 3].index)

# There seems to be an incorrect entry, drop it
nickel_data = nickel_data.drop(
    nickel_data[nickel_data['Heat treatment 2 Temperature'] > 7599].index)

# Drop the names column
nickel_data = nickel_data.drop(columns="Name")

nickel_data.head()


,"Tensile Strength, Yield",Heat treatment 1 Temperature,Heat treatment 1 Time,Heat treatment 2 Temperature,Heat treatment 2 Time,Heat treatment 3 Temperature,Heat treatment 3 Time,Heat treatment 4 Temperature,Heat treatment 4 Time,Heat treatment 5 Temperature,...,Ta,Ti,V,W,Y,Th,O,Zr,"Tensile Strength, Ultimate",Elongation at Break
0,989.0,954.0,1.0,788.0,8.0,704.0,8,760.0,500,0,...,0.0,0.7,0.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN
1,NaN,1121.0,0.0,0.0,0.0,0.0,0,0.0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,845.0,NaN
2,NaN,1232.0,0.0,0.0,0.0,0.0,0,0.0,0,0,...,0.0,0,0.0,14.0,0.0,0.0,0.0,0.0,NaN,NaN
3,NaN,1010.0,4.0,843.0,4.0,760.0,16,0.0,0,0,...,0.0,3,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
5,1074.0,1149.0,1.0,700.0,16.0,0.0,0,0.0,0,0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,26.4


## Train UTS Model

In [114]:
uts_data = nickel_data.dropna(subset="Tensile Strength, Ultimate")
uts_data = uts_data.drop(
    columns=['Tensile Strength, Yield', 'Elongation at Break'])
uts_data = uts_data.fillna(0)

uts_train_data = uts_data.sample(frac=0.8, random_state=6225)
uts_test_data = uts_data.drop(uts_train_data.index)

uts_data.head()

,Heat treatment 1 Temperature,Heat treatment 1 Time,Heat treatment 2 Temperature,Heat treatment 2 Time,Heat treatment 3 Temperature,Heat treatment 3 Time,Heat treatment 4 Temperature,Heat treatment 4 Time,Heat treatment 5 Temperature,Heat treatment 5 Time,...,Si,Ta,Ti,V,W,Y,Th,O,Zr,"Tensile Strength, Ultimate"
1,1121.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,...,0.40,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,845.0
11,1160.0,4.0,1080.0,4.0,850.0,24,760.0,16,0,0,...,0.02,0.0,3.45,0.0,0.0,0.0,0.0,0.0,0.0,1466.0
13,1121.0,1.0,843.0,4.0,718.0,8,621.0,8,0,0,...,0.06,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,816.0
24,1288.0,4.0,1079.0,4.0,871.0,16,0.0,0,0,0,...,0.00,12.0,1.5,0.0,4.0,0.0,0.0,0.0,0.0,1140.0
25,982.0,1.0,843.0,3.0,718.0,8,621.0,8,0,0,...,0.09,0.0,1.68,0.0,0.0,0.0,0.0,0.0,0.0,1372.0


In [115]:
uts_train_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Heat treatment 1 Temperature,642.0,1071.706300,114.885966,704.0,982.0000,1100.000000,1160.0000,1316.000
Heat treatment 1 Time,642.0,10.236760,52.849933,0.0,1.0000,2.000000,4.0000,1175.000
Heat treatment 2 Temperature,642.0,711.481828,339.944815,0.0,718.0000,788.000000,855.2500,1185.000
Heat treatment 2 Time,642.0,16.696262,136.246460,0.0,1.0000,4.000000,8.0000,2000.000
Heat treatment 3 Temperature,642.0,460.876082,383.645550,0.0,0.0000,621.111111,760.0000,1185.000
Heat treatment 3 Time,642.0,17.267913,82.523309,0.0,0.0000,8.000000,16.0000,760.000
Heat treatment 4 Temperature,642.0,113.704915,264.401094,0.0,0.0000,0.000000,0.0000,871.000
Heat treatment 4 Time,642.0,31.904984,211.272688,0.0,0.0000,0.000000,0.0000,1500.000
Heat treatment 5 Temperature,642.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0000,0.000
Heat treatment 5 Time,642.0,0.000000,0.000000,0.0,0.0000,0.000000,0.0000,0.000


In [116]:
uts_train_features = uts_train_data.copy()
uts_test_features = uts_test_data.copy()

uts_train_labels = uts_train_features.pop("Tensile Strength, Ultimate")
uts_test_labels = uts_test_features.pop("Tensile Strength, Ultimate")

In [117]:
uts_normalizer = tf.keras.layers.Normalization(axis=-1)
uts_normalizer.adapt(np.array(uts_train_features).astype('float32'))

In [118]:
first = np.array(uts_train_features[:1]).astype('float32')

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', uts_normalizer(first).numpy())


First example: [[1191.      4.    843.      4.    760.      4.      0.      0.      0.
     0.      3.64    0.03    0.06   17.66   12.47    0.      0.      0.
     0.      0.      0.      3.96    0.      1.9    55.95    0.      0.
     0.      0.      0.      4.28    0.      0.02    0.      0.      0.
     0.03]]

Normalized: [[ 1.04 -0.12  0.39 -0.09  0.78 -0.16 -0.43 -0.15  0.    0.    0.78  1.12
   0.06  1.13 -0.72 -0.34 -0.64 -0.06  0.    2.42 -0.38  0.25 -0.12  0.14
   0.01 -0.29  0.    0.   -0.53 -0.38  1.3  -0.14 -0.5   0.    0.    0.
   0.07]]


## Train Yield Model

In [119]:
yield_data = nickel_data.dropna(subset='Tensile Strength, Yield')
yield_data = yield_data.drop(
    columns=["Tensile Strength, Ultimate", 'Elongation at Break'])
yield_data = yield_data.fillna(0)

yield_train_data = yield_data.sample(frac=0.8, random_state=1234)
yield_test_data = yield_data.drop(yield_train_data.index)

yield_data.head()

,"Tensile Strength, Yield",Heat treatment 1 Temperature,Heat treatment 1 Time,Heat treatment 2 Temperature,Heat treatment 2 Time,Heat treatment 3 Temperature,Heat treatment 3 Time,Heat treatment 4 Temperature,Heat treatment 4 Time,Heat treatment 5 Temperature,...,Ru,Si,Ta,Ti,V,W,Y,Th,O,Zr
0,989.0,954.0,1.0,788.0,8.0,704.0,8,760.0,500,0,...,0.0,0.00,0.0,0.7,0.0,1.00,0.0,0.0,0.0,0.000
5,1074.0,1149.0,1.0,700.0,16.0,0.0,0,0.0,0,0,...,0.0,0.00,0.0,0,0.0,0.00,0.0,0.0,0.0,0.000
12,260.0,1177.0,1.0,816.0,4.0,0.0,0,0.0,0,0,...,0.0,0.50,0.0,0,0.0,2.50,0.0,0.0,0.0,0.000
14,1581.0,1175.0,2.0,760.0,8.0,649.0,24,0.0,0,0,...,0.0,0.00,0.0,5,0.0,1.25,0.0,0.0,0.0,0.035
16,518.0,982.0,1.0,622.0,4.0,0.0,0,0.0,0,0,...,0.0,0.07,0.0,0.27,0.0,0.00,0.0,0.0,0.0,0.000


In [120]:
yield_train_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
"Tensile Strength, Yield",746.0,810.085255,305.050689,40.0,617.250,835.5000,1018.750,1654.000000
Heat treatment 1 Temperature,746.0,1201.092225,503.153044,704.0,1020.000,1121.0000,1175.000,3812.400000
Heat treatment 1 Time,746.0,4.580764,8.742323,0.0,1.000,2.0000,4.000,50.000000
Heat treatment 2 Temperature,746.0,786.960322,491.427494,0.0,704.000,799.0000,871.000,2327.400000
Heat treatment 2 Time,746.0,23.741957,172.520064,0.0,2.000,6.0000,8.000,2000.000000
Heat treatment 3 Temperature,746.0,466.291421,517.581924,0.0,0.000,621.0000,760.000,2012.400000
Heat treatment 3 Time,746.0,12.404826,64.688098,0.0,0.000,8.0000,16.000,1000.000000
Heat treatment 4 Temperature,746.0,78.270777,221.065338,0.0,0.000,0.0000,0.000,843.000000
Heat treatment 4 Time,746.0,6.372654,50.450593,0.0,0.000,0.0000,0.000,1000.000000
Heat treatment 5 Temperature,746.0,5.093834,62.052624,0.0,0.000,0.0000,0.000,760.000000


In [121]:
yield_train_features = yield_train_data.copy()
yield_test_features = yield_test_data.copy()

yield_train_labels = yield_train_features.pop('Tensile Strength, Yield')
yield_test_labels = yield_test_features.pop('Tensile Strength, Yield')

yield_train_features.head()

,Heat treatment 1 Temperature,Heat treatment 1 Time,Heat treatment 2 Temperature,Heat treatment 2 Time,Heat treatment 3 Temperature,Heat treatment 3 Time,Heat treatment 4 Temperature,Heat treatment 4 Time,Heat treatment 5 Temperature,Heat treatment 5 Time,...,Ru,Si,Ta,Ti,V,W,Y,Th,O,Zr
575,1175.0,2.0,760.0,8.0,649.0,24,0.0,0,0,0,...,0.0,0.00,0.00,5,0.0,1.25,0.0,0.0,0.0,0.035
2226,3092.4,1.0,2327.4,8.0,2012.4,8,0.0,0,0,0,...,0.0,0.60,0.00,0.37,0.0,0.00,0.0,0.0,0.0,0.000
2357,1052.0,1.0,816.0,4.0,0.0,0,0.0,0,0,0,...,0.0,0.00,4.02,2.07,0.0,2.17,0.0,0.0,0.0,0.000
97,3452.4,1.0,2327.4,8.0,2012.4,8,0.0,0,0,0,...,0.0,0.61,0.00,1.45,0.0,0.00,0.0,0.0,0.0,0.000
1580,1168.0,2.0,0.0,1121.0,760.0,8,649.0,24,0,0,...,0.0,0.00,0.00,5.04,0.0,1.20,0.0,0.0,0.0,0.000


In [122]:
yield_normalizer = tf.keras.layers.Normalization(axis=-1)
yield_normalizer.adapt(np.array(yield_train_features).astype('float32'))


In [123]:
first = np.array(yield_train_features[:1]).astype('float32')

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', yield_normalizer(first).numpy())

First example: [[1175.      2.    760.      8.    649.     24.      0.      0.      0.
     0.      2.5     0.02    0.03   14.75   16.      0.      0.      0.
     0.      0.      0.      3.      0.      0.     57.41    0.      0.
     0.      0.      0.      5.      0.      1.25    0.      0.      0.
     0.04]]

Normalized: [[-0.05 -0.3  -0.05 -0.09  0.35  0.18 -0.35 -0.13 -0.08 -0.08  0.24  0.66
  -0.72  0.79 -0.   -0.04 -0.62 -0.29 -0.11 -0.08 -0.41 -0.24 -0.11 -0.75
   0.11 -0.32  0.    0.   -0.54 -0.39  1.84 -0.11 -0.03 -0.14 -0.07 -0.15
   0.5 ]]


## Train Elongation Model

In [124]:
elong_data = nickel_data.dropna(subset='Elongation at Break')
elong_data = elong_data.drop(
    columns=["Tensile Strength, Ultimate", 'Tensile Strength, Yield'])
elong_data = elong_data.fillna(0)

elong_train_data = elong_data.sample(frac=0.8, random_state=4321)
elong_test_data = elong_data.drop(elong_train_data.index)

elong_data.head()

,Heat treatment 1 Temperature,Heat treatment 1 Time,Heat treatment 2 Temperature,Heat treatment 2 Time,Heat treatment 3 Temperature,Heat treatment 3 Time,Heat treatment 4 Temperature,Heat treatment 4 Time,Heat treatment 5 Temperature,Heat treatment 5 Time,...,Si,Ta,Ti,V,W,Y,Th,O,Zr,Elongation at Break
5,1149.0,1.0,700.0,16.0,0.0,0,0.0,0,0,0,...,0.00,0.0,0,0.0,0.00,0.0,0.0,0.0,0.000,26.4
12,1177.0,1.0,816.0,4.0,0.0,0,0.0,0,0,0,...,0.50,0.0,0,0.0,2.50,0.0,0.0,0.0,0.000,30.0
14,1175.0,2.0,760.0,8.0,649.0,24,0.0,0,0,0,...,0.00,0.0,5,0.0,1.25,0.0,0.0,0.0,0.035,17.3
16,982.0,1.0,622.0,4.0,0.0,0,0.0,0,0,0,...,0.07,0.0,0.27,0.0,0.00,0.0,0.0,0.0,0.000,47.0
20,1160.0,4.0,1080.0,4.0,850.0,24,760.0,16,0,0,...,0.04,0.0,3.57,0.0,0.00,0.0,0.0,0.0,0.000,2.6


In [125]:
elong_train_data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Heat treatment 1 Temperature,638.0,1220.120376,547.281675,704.0,1020.00000,1121.0000,1169.5000,3812.400000
Heat treatment 1 Time,638.0,4.666144,9.088142,0.0,1.00000,2.0000,4.0000,50.000000
Heat treatment 2 Temperature,638.0,807.923824,516.221934,0.0,718.00000,800.0000,955.0000,2327.400000
Heat treatment 2 Time,638.0,26.820533,186.395076,0.0,2.00000,6.0000,8.0000,2000.000000
Heat treatment 3 Temperature,638.0,480.122884,538.447420,0.0,0.00000,621.0000,760.0000,2012.400000
Heat treatment 3 Time,638.0,13.722571,69.839038,0.0,0.00000,8.0000,16.0000,1000.000000
Heat treatment 4 Temperature,638.0,82.036050,224.581313,0.0,0.00000,0.0000,0.0000,843.000000
Heat treatment 4 Time,638.0,3.047022,39.780177,0.0,0.00000,0.0000,0.0000,1000.000000
Heat treatment 5 Temperature,638.0,3.573668,52.033259,0.0,0.00000,0.0000,0.0000,760.000000
Heat treatment 5 Time,638.0,0.037618,0.547719,0.0,0.00000,0.0000,0.0000,8.000000


In [126]:
elong_train_features = elong_train_data.copy()
elong_test_features = elong_test_data.copy()

elong_train_labels = elong_train_features.pop('Elongation at Break')
elong_test_labels = elong_test_features.pop('Elongation at Break')


In [127]:
elong_normalizer = tf.keras.layers.Normalization(axis=-1)
elong_normalizer.adapt(np.array(elong_train_features).astype('float32'))


In [128]:
first = np.array(elong_train_features[:1]).astype('float32')

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', elong_normalizer(first).numpy())


First example: [[1024.     1.5  941.     4.   843.     7.     0.     0.     0.     0.
     0.5    0.     0.     0.    18.5    0.    18.5    0.     0.     0.
     0.     3.     0.     5.    53.4    0.     0.     0.     0.     0.
     1.     0.     0.     0.     0.     0.     0. ]]

Normalized: [[-0.36 -0.35  0.26 -0.12  0.67 -0.1  -0.37 -0.08 -0.07 -0.07 -0.89 -0.61
  -1.37 -1.08  0.49 -0.06  0.91 -0.3  -0.1   0.   -0.42 -0.17 -0.13  1.59
  -0.21 -0.3   0.    0.   -0.57 -0.38 -0.71 -0.12 -0.51 -0.14 -0.09 -0.17
  -0.68]]
